In [461]:
import pandas as pd
import numpy as np
from itertools import pairwise

In [462]:
np.set_printoptions(precision=3, floatmode='fixed', suppress=True)
pd.options.display.precision = 3
dtype = np.float32

In [463]:
headers = np.dtype( [('r', dtype), ('alpha', dtype)] )
data = np.array(
    [
        (100., np.deg2rad(30)),
        ( 90., np.deg2rad(40)),
        (110., np.deg2rad(20)),
    ],
    # dtype=headers
)
print(data)

[[100.000   0.524]
 [ 90.000   0.698]
 [110.000   0.349]]


In [464]:
def st(array):
    if isinstance(array, np.ndarray):
        assert array.shape == (2,), 'array shape must be (2,)'
    else:
        assert len(array) == 2
    result = array[0] * np.tan( array[1]/2 )
    return result
st_vec = lambda matrix: np.apply_along_axis(st, 1, matrix)
st_vec(data)

array([26.795, 32.757, 19.396])

In [465]:
np.sum(data, axis=0)

array([300.000,   1.571])

In [466]:
def diag(array, inverse = True):
    llist = []
    if inverse: array = array[::-1]
    for pair in pairwise(array):
        pair_a = np.array(pair)
        sum_ = np.sum(st_vec(pair_a))
        sum_a = np.sin(np.sum(np.array(pair_a), axis=0)[1])
        llist.append(sum_ * np.sin(pair_a[0,1]) / sum_a)
    return np.array(llist)
diag(data)

array([20.597, 40.736])

In [467]:
def pc_calc(data, pi=0):
    acc_inv = diag(data, inverse=True)
    diff_ = np.sum(st_vec(data)[:2]) + acc_inv[0]
    ang_pi = np.sum(data[:,1])
    ang_a = (np.pi - (data[0,1] + ang_pi))
    diff = (diff_ * np.sin(ang_a) / np.sin(ang_pi)) + st_vec(data)[0]
    return pi - diff
pc_calc(data, pi=14_000)

13903.793852415718

In [468]:
def p_pc(data, pi=0):
    pc_ = pc_calc(data, pi)
    prod_ = np.prod(data, axis=1)
    acc_ = np.add.accumulate(prod_)
    return np.insert(acc_ + pc_, 0, pc_, axis=0)
p_pc(data, pi=14_000).reshape(-1,1)

array([[13903.794],
       [13956.154],
       [14018.986],
       [14057.383]])

In [470]:
p_pc(data, pi=14_000).reshape(-1,1).shape

(4, 1)

In [471]:
_ = np.concatenate((np.array((0,0)).reshape(1,-1), data), axis=0)
_ = np.concatenate((_, p_pc(data, pi=14_000).reshape(-1,1)), axis=1)
_ = pd.DataFrame(_, columns=['Radios', 'Angulos', 'Cota Ac.'])
_.apply({'Radios': lambda x: x, 'Angulos': np.rad2deg, 'Cota Ac.': lambda x: x})
# print(pc_calc(data, pi=14_000))

,Radios,Angulos,Cota Ac.
0,0.0,0.0,13903.794
1,100.0,30.0,13956.154
2,90.0,40.0,14018.986
3,110.0,20.0,14057.383


In [501]:
def gen_marks(array, prec = 1):
    step = 10 ** prec
    start = (array[0] // 10**prec) * step
    end = ((array[-1] // 10**prec) + 2) * step
    arange = np.arange(start, end, step,)
    return np.unique(np.sort(np.concatenate([arange, array])), axis=0)

In [502]:
gen_marks(_['Cota Ac.'].values)

array([13900.000, 13903.794, 13910.000, 13920.000, 13930.000, 13940.000,
       13950.000, 13956.154, 13960.000, 13970.000, 13980.000, 13990.000,
       14000.000, 14010.000, 14018.986, 14020.000, 14030.000, 14040.000,
       14050.000, 14057.383, 14060.000])

In [ ]:
# to do
# for mark in gen_marks(_['Cota Ac.'].values):
    